In [1]:
#Import Python Libraries
import pandas as pd
import matplotlib.pyplot as plt



In [5]:
# Load the dataset containing the range information and crop labels
dataset = pd.read_csv('Crop_and_fertilizer dataset.csv')


In [6]:
dataset.head()


,Soil_color,Nitrogen,Phosphorus,Potassium,pH,Crop,Fertilizer
0,Black,75,50,100,6.5,Sugarcane,Urea
1,Black,80,50,100,6.5,Sugarcane,Urea
2,Black,85,50,100,6.5,Sugarcane,Urea
3,Black,90,50,100,6.5,Sugarcane,Urea
4,Black,95,50,100,6.5,Sugarcane,Urea


In [7]:
dataset


,Soil_color,Nitrogen,Phosphorus,Potassium,pH,Crop,Fertilizer
0,Black,75,50,100,6.5,Sugarcane,Urea
1,Black,80,50,100,6.5,Sugarcane,Urea
2,Black,85,50,100,6.5,Sugarcane,Urea
3,Black,90,50,100,6.5,Sugarcane,Urea
4,Black,95,50,100,6.5,Sugarcane,Urea
...,...,...,...,...,...,...,...
2916,Black,130,80,150,7.0,Sugarcane,MOP
2917,Black,135,80,150,7.0,Sugarcane,MOP
2918,Black,140,80,150,7.0,Sugarcane,MOP
2919,Black,145,80,150,7.0,Sugarcane,MOP


In [8]:
dataset.shape

(2921, 7)

In [9]:
dataset.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2921 entries, 0 to 2920
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Soil_color  2921 non-null   object 
 1   Nitrogen    2921 non-null   int64  
 2   Phosphorus  2921 non-null   int64  
 3   Potassium   2921 non-null   int64  
 4   pH          2921 non-null   float64
 5   Crop        2921 non-null   object 
 6   Fertilizer  2921 non-null   object 
dtypes: float64(1), int64(3), object(3)
memory usage: 159.9+ KB


In [10]:
dataset.duplicated().sum()

34

In [11]:
dataset.describe()


,Nitrogen,Phosphorus,Potassium,pH
count,2921.000000,2921.000000,2921.000000,2921.000000
mean,109.630264,59.698733,75.448477,6.754707
std,32.180999,15.169532,36.187594,0.639857
min,20.000000,15.000000,20.000000,5.500000
25%,90.000000,50.000000,50.000000,6.500000
50%,120.000000,60.000000,60.000000,6.500000
75%,135.000000,70.000000,100.000000,7.000000
max,150.000000,90.000000,150.000000,8.500000


In [17]:
import ipywidgets as widgets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier


In [29]:
# Create combo box for District_name

# district_widget = widgets.Combobox(
#     options=tuple(dataset['District_Name'].unique()),
#     description='District:',
#     placeholder='Select district',
#     ensure_option=True
# )
# Create an empty soil color widget initially
soil_color_widget = widgets.Combobox(
     options=tuple(dataset['Soil_color'].unique()),
    description='Soil Color:',
    placeholder='Select soil color',
    ensure_option=True
)

nitrogen_widget = widgets.Combobox(
    description='Nitrogen:',
    placeholder='Select nitrogen value',
    ensure_option=True
)

phosphorus_widget = widgets.Combobox(
    description='Phosphorus:',
    placeholder='Select phosphorus value',
    ensure_option=True
)

potassium_widget = widgets.Combobox(
    description='Potassium:',
    placeholder='Select potassium value',
    ensure_option=True
)

ph_widget = widgets.Combobox(
    description='pH:',
    placeholder='Select pH value',
    ensure_option=True
)




recommend_widget = widgets.Output()


# Define the observer function to update soil color options
def update_soil_color_options(change):
    soil_color = change.new
    if soil_color:
        soil_colors = dataset[dataset['PH'] == district]['Soil_color'].unique()
        soil_color_widget.options = tuple(soil_colors)
    else:
        soil_color_widget.options = ()

# Define the observer function to update nitrogen options
def update_nitrogen_options(change):
    soil_color = soil_color_widget.value
    if soil_color:
        nitrogen_values = dataset[(dataset['Soil_color'] == soil_color)]['Nitrogen'].unique()
        nitrogen_values = [str(value) for value in nitrogen_values]  # Convert to Unicode strings
        nitrogen_widget.options = tuple(nitrogen_values)
    else:
        nitrogen_widget.options = ()
        
def update_phosphorus_options(change):
    soil_color = soil_color_widget.value
    if soil_color:
        phosphorus_values = dataset[ (dataset['Soil_color'] == soil_color)]['Phosphorus'].unique()
        phosphorus_values = [str(value) for value in phosphorus_values]  # Convert to Unicode strings
        phosphorus_widget.options = tuple(phosphorus_values)
    else:
        phosphorus_widget.options = ()
        
def update_potassium_options(change):
    soil_color = soil_color_widget.value
    if soil_color:
        potassium_values = dataset[ (dataset['Soil_color'] == soil_color)]['Potassium'].unique()
        potassium_values = [str(value) for value in potassium_values]  # Convert to Unicode strings
        potassium_widget.options = tuple(potassium_values)
    else:
        potassium_widget.options = ()
        
def update_ph_options(change):
    soil_color = soil_color_widget.value
    if soil_color:
        ph_values = dataset[(dataset['Soil_color'] == soil_color)]['pH'].unique()
        ph_values = [str(value) for value in ph_values]  # Convert to Unicode strings
        ph_widget.options = tuple(ph_values)
    else:
        ph_widget.options = ()
        



# Train the model
def train_model(change):
    # Get the selected values from the combo boxes
 
    soil_color = soil_color_widget.value
    nitrogen = float(nitrogen_widget.value)
    phosphorus = float(phosphorus_widget.value)
    potassium = float(potassium_widget.value)
    pH = float(ph_widget.value)
   
   
    input_data = pd.DataFrame(
        [[nitrogen, phosphorus, potassium, pH, soil_color]],
        columns=['Nitrogen', 'Phosphorus', 'Potassium', 'pH', 'Soil_color']
    )
    
    # Perform one-hot encoding for District_Name and Soil_color columns
    encoder = OneHotEncoder(handle_unknown='ignore')
    X_encoded = encoder.fit_transform(dataset[['Soil_color']])
    input_data_encoded = encoder.transform(input_data[['Soil_color']])

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_encoded, dataset['Crop'], test_size=0.2, random_state=42)

    # Train the random forest model
    model_crop = RandomForestClassifier(n_estimators=100, random_state=42)
    model_crop.fit(X_train, y_train)

    # Make predictions
    predicted_crop = model_crop.predict(input_data_encoded)

    # Find the fertilizer associated with the recommended crop
    recommended_fertilizer = dataset[dataset['Crop'] == predicted_crop[0]]['Fertilizer'].values[0]
    # Find the corresponding link for the predicted crop and fertilizer
    #link = dataset[(dataset['Crop'] == predicted_crop.iloc[0]) & (dataset['Fertilizer'] == recommended_fertilizer)]['Link'].values[0]
    #-link = dataset[(dataset['Crop'] == pd.Series(predicted_crop)[0]) & (dataset['Fertilizer'] == recommended_fertilizer)]['Link'].values[0]
    with recommend_widget:
        recommend_widget.clear_output()
        print("Recommended Crop:", predicted_crop[0])
        print("Recommended Fertilizer:", recommended_fertilizer)
       #- print("Link:", link)

# Define the observer function to update soil color options

soil_color_widget.observe(update_nitrogen_options, names='value')



soil_color_widget.observe(update_phosphorus_options, names='value')


soil_color_widget.observe(update_potassium_options, names='value')

soil_color_widget.observe(update_ph_options, names='value')







# Create the button widget
button = widgets.Button(description='Train Model')
# Add the train_model function as an observer to the 'on_click' event of the button
button.on_click(train_model)

# Display the widgets
widgets.VBox([soil_color_widget, nitrogen_widget,phosphorus_widget,potassium_widget,ph_widget,button,recommend_widget])

